In [2]:
import cv2 as cv
import numpy as np
from keras.models import load_model
model=load_model("./mask.h5")
results={0:'mask',1:'no mask'}
GR_dict={1:(0,0,255),0:(0,255,0)}
rect_size = 5
cap = cv.VideoCapture(0)
haarcascade = cv.CascadeClassifier(r'F:\codes\vs code pgms\haarcascades\haarcascade_frontalface_default.xml')
while True:
    (rval, im) = cap.read()
    im=cv.flip(im,1,1) 
    rerect_size = cv.resize(im, (im.shape[1]// rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv.resize(face_img,(224,224))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
      
        cv.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv.putText(im, results[label], (x, y-10),cv.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv.imshow('LIVE',   im)
    key = cv.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv.destroyAllWindows()